In [1]:
import datasets
import numpy as np
import pandas as pd

/home/felix/.local/share/virtualenvs/Comparing-Measures-of-LLM-Truthfulness-c5kw3004/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('LieDetector/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
from simpleTQA import SIMPLE_TQA_QUESTION_PREFIX, SIMPLE_TQA_ANSWER_PREFIX

In [17]:
np.random.seed(0)
tqa = datasets.load_dataset("truthful_qa", "multiple_choice")
tqa = pd.DataFrame(tqa["validation"])

questions = []
answers = []
full_prompts = []

for _, row in tqa.iterrows():
    choices = row["mc1_targets"]["choices"]
    correct_choice_idx = row["mc1_targets"]["labels"].index(1)
    false_choice_indices = list(range(len(choices))) 
    false_choice_indices.remove(correct_choice_idx)
    false_choice_idx = np.random.choice(false_choice_indices)
    
    question_with_true_answer = f"Question: {row['question']}\nAnswer: {choices[correct_choice_idx]}"
    full_prompt_true_answer = f"{SIMPLE_TQA_QUESTION_PREFIX}{question_with_true_answer}\n{SIMPLE_TQA_ANSWER_PREFIX}" 
    questions.append(question_with_true_answer)
    answers.append("True")
    full_prompts.append(full_prompt_true_answer)
    
    question_with_false_answer = f"Question: {row['question']}\nAnswer: {choices[false_choice_idx]}"
    full_prompt_false_answer = f"{SIMPLE_TQA_QUESTION_PREFIX}{question_with_false_answer}\n{SIMPLE_TQA_ANSWER_PREFIX}" 
    questions.append(question_with_false_answer)
    answers.append("False")
    full_prompts.append(full_prompt_false_answer)

simple_tqa = pd.DataFrame({
    "question": questions,
    "answer": answers,
    "full_prompt": full_prompts,
})

In [18]:
simple_tqa.to_csv("data/simple_tqa.csv", index=False)